In [ ]:
from nltk.util import ngrams
from collections import defaultdict
from collections import OrderedDict
import string
import time
import gc
from math import log10
start_time = time.time()

In [ ]:

emails_tokens=pd.read_pickle('./emailTokens')
emails=pd.DataFrame(emails_tokens)
df=emails
len(df)

In [ ]:
train=emails[0:258700]
test = emails[336310:]
valid=emails[258700:336310]
test.to_csv('testing_set_2.csv')


In [ ]:

def loadCorpus(df, bi_dict, tri_dict, quad_dict, vocab_dict):

    w1 = ''    #for storing the 3rd last word to be used for next token set
    w2 = ''    #for storing the 2nd last word to be used for next token set
    w3 = ''    #for storing the last word to be used for next token set
    token = []
    word_len = 0
    for token in df['message_tokens']:

        word_len = word_len + len(token)  
        if not token:
            continue
        if w3!= '':
            token.insert(0,w3)

        temp0 = list(ngrams(token,2))
        if w2!= '':
            token.insert(0,w2)
        temp1 = list(ngrams(token,3))
        if w1!= '':
            token.insert(0,w1)
        for word in token:
            if word not in vocab_dict:
                vocab_dict[word] = 1
            else:
                vocab_dict[word]+= 1
        temp2 = list(ngrams(token,4))
        for t in temp0:
            sen = ' '.join(t)
            bi_dict[sen] += 1
        for t in temp1:
            sen = ' '.join(t)
            tri_dict[sen] += 1
        for t in temp2:
            sen = ' '.join(t)
            quad_dict[sen] += 1
        n = len(token)
        if (n -3) >= 0:
            w1 = token[n -3]
        if (n -2) >= 0:
            w2 = token[n -2]
        if (n -1) >= 0:
            w3 = token[n -1]
    return word_len

In [ ]:

def findQuadgramProbAdd1(vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict):
    i = 0
    V = len(vocab_dict)
    for quad_sen in quad_dict:
        quad_token = quad_sen.split()
        tri_sen = ' '.join(quad_token[:3])
        prob = ( quad_dict[quad_sen] + 1 ) / ( tri_dict[tri_sen] + V)
        if tri_sen not in quad_prob_dict:
            quad_prob_dict[tri_sen] = []
            quad_prob_dict[tri_sen].append([prob,quad_token[-1]])
        else:
            quad_prob_dict[tri_sen].append([prob,quad_token[-1]])
    prob = None
    quad_token = None
    tri_sen = None

In [ ]:
def findTrigramProbAdd1(vocab_dict, bi_dict, tri_dict, tri_prob_dict):
    V = len(vocab_dict)
    for tri in tri_dict:
        tri_token = tri.split()
        bi_sen = ' '.join(tri_token[:2])
        prob = ( tri_dict[tri] + 1 ) / ( bi_dict[bi_sen] + V)
        if bi_sen not in tri_prob_dict:
            tri_prob_dict[bi_sen] = []
            tri_prob_dict[bi_sen].append([prob,tri_token[-1]])
        else:
            tri_prob_dict[bi_sen].append([prob,tri_token[-1]])
    prob = None
    tri_token = None
    bi_sen = None

In [ ]:
def findBigramProbAdd1(vocab_dict, bi_dict, bi_prob_dict):
    V = len(vocab_dict)
    for bi in bi_dict:
        bi_token = bi.split()
        unigram = bi_token[0]
        prob = ( bi_dict[bi] + 1 ) / ( vocab_dict[unigram] + V)
        if unigram not in bi_prob_dict:
            bi_prob_dict[unigram] = []
            bi_prob_dict[unigram].append([prob,bi_token[-1]])
        else:
            bi_prob_dict[unigram].append([prob,bi_token[-1]])
    prob = None
    bi_token = None
    unigram = None

In [ ]:
def estimateParameters(valid,token_len, vocab_dict, bi_dict, tri_dict, quad_dict):
    max_prob = -9999999999999999999.0
    curr_prob = 0.0
    parameters = [0.0,0.0,0.0,0.0]
    i = 1
    held_out_data = valid['message_tokens']
    for quad in held_out_data:
        quad_token_heldout = list(ngrams(quad,4))
        l1 = 0
        l4 = 0
        while l1 <= 1.0:
            l2 = 0
            while l2 <= 1.0:
                l3 = 0
                while l3 <= 1.0:
                    if l1 == 0 and l2 == 0 and l3 == 0 or ((l1+l2+l3)>1):
                        l3 += 0.1
                        i += 1
                        continue
                    l4 = 1- (l1 + l2 + l3)      
                    curr_prob = 0
                    qc = [0]
                    bc = [0]
                    tc = [0]
                    for quad in quad_token_heldout:
                        curr_prob += log10( interpolatedProbability(quad,token_len, vocab_dict, bi_dict, tri_dict, 
                                                                    quad_dict,qc,bc,tc,l1, l2, l3, l4) )
                    if curr_prob > max_prob:
                        max_prob = curr_prob
                        parameters[0] = l1
                        parameters[1] = l2
                        parameters[2] = l3
                        parameters[3] = l4
                    l3 += 0.1
                    i += 1

                l2 += 0.1
            l1 += 0.1
    return parameters

In [ ]:
def interpolatedProbability(quad_token,token_len, vocab_dict, bi_dict, tri_dict, quad_dict, qc, tc, bc,
                            l1 = 0.25, l2 = 0.25, l3 = 0.25 , l4 = 0.25):
    V = len(vocab_dict)
    
    sen = ' '.join(quad_token)
    prob = (   
              l1*((quad_dict[sen] + 1)/ (tri_dict[' '.join(quad_token[0:3])] + V)) 
            + l2*((tri_dict[' '.join(quad_token[1:4])] + 1) / (bi_dict[' '.join(quad_token[1:3])] + V)) 
            + l3*((bi_dict[' '.join(quad_token[2:4])] + 1) / (vocab_dict[quad_token[2]] + V)) 
            + l4*((vocab_dict[quad_token[3]] + 1) / (token_len + V))
           )
    
    if sen  in quad_dict:
        qc[0] += 1
    if ' '.join(quad_token[1:4]) in tri_dict:
        tc[0] += 1
    if ' '.join(quad_token[2:4])  in bi_dict:
        bc[0] += 1    
    if prob <= 0:
        return 1
    
    return prob

In [ ]:

def sortProbWordDict(bi_prob_dict, tri_prob_dict, quad_prob_dict):
    #sort bigram dict
    for key in bi_prob_dict:
        if len(bi_prob_dict[key])>1:
            bi_prob_dict[key] = sorted(bi_prob_dict[key],reverse = True)
    
    #sort trigram dict
    for key in tri_prob_dict:
        if len(tri_prob_dict[key])>1:
            tri_prob_dict[key] = sorted(tri_prob_dict[key],reverse = True)
    
    #sort quadgram dict
    for key in quad_prob_dict:
        if len(quad_prob_dict[key])>1:
            quad_prob_dict[key] = sorted(quad_prob_dict[key],reverse = True)[:2]

In [ ]:

def chooseWords(sen, bi_prob_dict, tri_prob_dict, quad_prob_dict):
    word_choice = []
    token = sen.split()
    if token[-1] in bi_prob_dict:
        word_choice +=  bi_prob_dict[token[-1]][:1]
        #print('Word Choice bi dict')
    if ' '.join(token[1:]) in tri_prob_dict:
        word_choice +=  tri_prob_dict[' '.join(token[1:])][:1]
        #print('Word Choice tri_dict')
    if ' '.join(token) in quad_prob_dict:
        word_choice += quad_prob_dict[' '.join(token)][:1]
        #print('Word Choice quad_dict')
    
    return word_choice

In [ ]:

def doInterpolatedPredictionAdd1(sen, bi_dict, tri_dict, quad_dict, 
                             vocab_dict,token_len, word_choice, param):
    pred = ''
    max_prob = 0.0
#     V = len(vocab_dict)
    V=67007
    #for each word choice find the interpolated probability and decide
    for word in word_choice:
        key = sen + ' ' + word[1]
        quad_token = key.split()
        
        prob = (   
                  param[0]*((quad_dict[key] + 1)/ (tri_dict[' '.join(quad_token[0:3])] + V)) 
                + param[1]*((tri_dict[' '.join(quad_token[1:4])] + 1) / (bi_dict[' '.join(quad_token[1:3])] + V)) 
                + param[2]*((bi_dict[' '.join(quad_token[2:4])] + 1) / (vocab_dict[quad_token[2]] + V)) 
                + param[3]*((vocab_dict[quad_token[3]] + 1) / (token_len + V))
               )
        
        if prob > max_prob:
            max_prob = prob
            pred = word
    #return only pred to get word with its prob
    if pred:
        return pred[1]
    else:
        return ''

In [ ]:

def computeTestScore(test_token,bi_dict, tri_dict, quad_dict, vocab_dict,
                     bi_prob_dict, tri_prob_dict, quad_prob_dict, token_len,param):
    
    
    #increment the score value if correct prediction is made else decrement its value
    score = 0
    wrong = 0
    total = 0
    for tokens in test_token:
        for sent in tokens:
            if(len(sent)<4):
                break
            sen_token = sent[:3]
            sen = " ".join(sen_token)
            correct_word = sent[3]
            word_choice = chooseWords(sen, bi_prob_dict, tri_prob_dict, quad_prob_dict)
            result = doInterpolatedPredictionAdd1(sen, bi_dict, tri_dict, quad_dict, vocab_dict,token_len, word_choice, param)

            if result == correct_word:
                score+=1
            else:
                wrong += 1

            total += 1

    print('Total Word Predictions: '+str(total) + '\n' +'Correct Predictions: '+str(score) +
                    '\n'+'Wrong Predictions: '+str(wrong) + '\n'+'acc:'+str((score/total)*100) )
    return score

In [ ]:

def computePerplexity(test_quadgrams,token_len,tri_dict,quad_dict,vocab_dict,prob_dict):
    
    perplexity = float(1.0)
    n = token_len
#     V = len(vocab_dict)
    V=67007
    for item in quad_dict:
        sen_token = item.split()
        tri_sen = ' '.join(sen_token[0:3])
        prob = (quad_dict[item] + 1) / (tri_dict[tri_sen] + V)
        perplexity = perplexity * ( prob**(1./n))
    with open('Test_Scores/add1_smoothing_score.txt','w') as w:
        w.write('\nPerplexity: '+str(perplexity))
    return perplexity

In [ ]:

def trainCorpus(valid,train_file,test_file,bi_dict,tri_dict,quad_dict,vocab_dict,prob_dict):
    score = 0
    
    #load the training corpus for the dataset
    token_len = loadCorpus(train_file, bi_dict, tri_dict, quad_dict, vocab_dict)
    print("---Processing Time for Corpus Loading: %s seconds ---" % (time.time() - start_time))

    start_time1 = time.time()

    param = estimateParameters(valid,token_len, vocab_dict, bi_dict, tri_dict, quad_dict)    
    #create trigram Probability Dictionary
    findTrigramProbAdd1(vocab_dict, bi_dict, tri_dict, tri_prob_dict)
    #create bigram Probability Dictionary
    findBigramProbAdd1(vocab_dict, bi_dict, bi_prob_dict)
    #create quadgram Probability Dictionary
    findQuadgramProbAdd1(vocab_dict, bi_dict, tri_dict, quad_dict, quad_prob_dict)
    #sort the probability dictionaries
    sortProbWordDict(bi_prob_dict, tri_prob_dict, quad_prob_dict)
    gc.collect()
    print("Preprocessing Time for Creating Probable Word Dict: %s seconds" % (time.time() - start_time1))

    test_quadgrams = list(ngrams(test['message_tokens'],4))
    start_time2 = time.time()
    score = computeTestScore(test_quadgrams,bi_dict, tri_dict, quad_dict, vocab_dict,
                     bi_prob_dict, tri_prob_dict, quad_prob_dict, token_len,param)
    print('Score:',score)
    print("Processing Time for computing score: %s seconds" % (time.time() - start_time2))

    start_time3 = time.time()
    perplexity = computePerplexity(test_quadgrams,token_len,tri_dict,quad_dict,vocab_dict,prob_dict)
    print('Perplexity:',perplexity)
    print("Processing Time for computing Perplexity: %s seconds" % (time.time() - start_time3))


In [ ]:
train_file = 'train.csv'
test_file = 'test.csv'
token_len = trainCorpus(valid,df,test_file,bi_dict,tri_dict,quad_dict,vocab_dict,quad_prob_dict)

### For Testing the Model


In [ ]:
import pandas as pd
test = pd.read_csv('testing_set_2.csv')

In [ ]:
vocab_dict = defaultdict(int)          
bi_dict = defaultdict(int)             
tri_dict = defaultdict(int)            
quad_dict = defaultdict(int)           
quad_prob_dict = defaultdict(list)             
tri_prob_dict = defaultdict(list)              
bi_prob_dict = defaultdict(list)              

In [ ]:
test_quadgrams = list(ngrams(test['message_tokens'],4))
import numpy as np
bi_dict=np.load("bi_dict.npy")  
tri_dict = np.load("tri_dict.npy")
quad_dict = np.load("quad_dict.npy")
vocab_dict=np.load("vocab_dict.npy")
bi_prob_dict=np.load("bi_prob_dict.npy")
tri_prob_dict=np.load("tri_prob_dict.npy")
quad_prob_dict=np.load("quad_prob_dict.npy")
token_len=64355494
#choose most probable words for prediction
start_time2 = time.time()
param=[0,0,0.1,0.9]
score = computeTestScore(test_quadgrams,bi_dict, tri_dict, quad_dict, vocab_dict,
                 bi_prob_dict, tri_prob_dict, quad_prob_dict, token_len,param)
print('Score:',score)
print("Processing Time for computing score: %s seconds" % (time.time() - start_time2))

start_time3 = time.time()
perplexity = computePerplexity(test_quadgrams,token_len,tri_dict,quad_dict,vocab_dict,prob_dict)
print('Perplexity:',perplexity)
print("Processing Time for computing Perplexity: %s seconds" % (time.time() - start_time3))
